In [1]:
!pip install scikit-learn pandas numpy
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split


In [2]:
!wget -nc http://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip -n ml-100k.zip
columns = ["user_id", "item_id", "rating", "timestamp"]
ratings = pd.read_csv("ml-100k/u.data", sep="\t", names=columns)
movie_cols = ["item_id", "title", "release_date", "video_release_date", "IMDb_URL",
              "unknown", "Action", "Adventure", "Animation", "Children's", "Comedy", "Crime",
              "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror", "Musical",
              "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"]
movies = pd.read_csv("ml-100k/u.item", sep="|", names=movie_cols, encoding="latin-1")
data = ratings.merge(movies[["item_id", "title"]], on="item_id")

print("Ratings Data Sample:")
print(data.head())


--2025-09-04 07:16:36--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.96.204
Connecting to files.grouplens.org (files.grouplens.org)|128.101.96.204|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://files.grouplens.org/datasets/movielens/ml-100k.zip [following]
--2025-09-04 07:16:36--  https://files.grouplens.org/datasets/movielens/ml-100k.zip
Connecting to files.grouplens.org (files.grouplens.org)|128.101.96.204|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  3.51MB/s    in 1.3s    

2025-09-04 07:16:38 (3.51 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml

In [3]:
user_item_matrix = data.pivot_table(index="user_id", columns="title", values="rating")

user_item_matrix_filled = user_item_matrix.fillna(0)

print("User-Item Matrix Shape:", user_item_matrix_filled.shape)


User-Item Matrix Shape: (943, 1664)


In [4]:
user_similarity = cosine_similarity(user_item_matrix_filled)
user_similarity_df = pd.DataFrame(user_similarity,
                                  index=user_item_matrix.index,
                                  columns=user_item_matrix.index)

print("User Similarity Matrix (sample):")
print(user_similarity_df.iloc[:5, :5])


User Similarity Matrix (sample):
user_id         1         2         3         4         5
user_id                                                  
1        1.000000  0.168937  0.048388  0.064561  0.379670
2        0.168937  1.000000  0.113393  0.179694  0.073623
3        0.048388  0.113393  1.000000  0.349781  0.021592
4        0.064561  0.179694  0.349781  1.000000  0.031804
5        0.379670  0.073623  0.021592  0.031804  1.000000


In [5]:
def recommend_movies(target_user, top_n=5):
    sim_scores = user_similarity_df[target_user].drop(target_user)
    similar_users = sim_scores.sort_values(ascending=False).index

    seen_movies = user_item_matrix.loc[target_user].dropna().index
    weighted_ratings = pd.Series(dtype=float)

    for user in similar_users:
        user_ratings = user_item_matrix.loc[user]

        user_ratings = user_ratings.drop(seen_movies, errors="ignore")
        weighted_ratings = weighted_ratings.add(user_ratings * sim_scores[user], fill_value=0)
    recommendation_scores = weighted_ratings / sim_scores.sum()
    top_recommendations = recommendation_scores.dropna().sort_values(ascending=False).head(top_n)
    return top_recommendations

print("Recommended Movies for User 10:")
print(recommend_movies(target_user=10, top_n=5))


Recommended Movies for User 10:
title
Return of the Jedi (1983)          2.625570
Empire Strikes Back, The (1980)    2.382551
Back to the Future (1985)          2.135831
Schindler's List (1993)            2.132099
Fugitive, The (1993)               2.104110
dtype: float64


In [6]:
def precision_at_k(test_data, k=5):
    precisions = []

    for user in test_data["user_id"].unique():
        user_data = test_data[test_data["user_id"] == user]
        relevant_items = set(user_data[user_data["rating"] >= 4]["item_id"].values)

        if len(relevant_items) == 0:
            continue
        recs = recommend_movies(user, top_n=k).index
        rec_items = data[data["title"].isin(recs)]["item_id"].values
        hits = len(set(rec_items) & relevant_items)
        precisions.append(hits / k)

    return np.mean(precisions)

train, test = train_test_split(data, test_size=0.2, random_state=42)

print("Precision@5:", precision_at_k(test, k=5))


Precision@5: 0.0


#Movie Recommendation System

This project was about building a system that recommends movies to users based on their preferences and similarity with other users. On the surface, it seems simple — “give me movies I might like” — but it turned out to be more nuanced than that. The main challenge was that not all users rate movies consistently, and most users haven’t seen every movie, so the dataset was very sparse. That means the system has to infer preferences from very limited information.

The dataset I used was the MovieLens 100K dataset, which has a mix of numerical and categorical data — ratings, user IDs, movie IDs, and movie titles. The first step was loading and exploring the data to understand its structure. I noticed that some movies were rated very frequently while others had very few ratings, which meant popularity could bias the recommendations if not handled carefully.

Creating the user-item matrix was straightforward conceptually, but dealing with the missing ratings required thought. I filled unrated movies with zeros when computing similarity, because ignoring them could make the similarity calculations misleading. Then I computed user similarity using cosine similarity, which measures how similar one user’s ratings are to another’s. I chose cosine similarity because it focuses on the pattern of ratings rather than the absolute rating values, which seemed more appropriate for this scenario.

After that, I needed a strategy to recommend movies that a user hadn’t seen yet. The approach I used was simple but effective: look at the users most similar to the target user, consider their ratings for movies the target user hasn’t rated, and then suggest the highest-rated unseen movies. It was interesting to see how users with similar tastes could reveal recommendations that weren’t obvious at first glance.

Evaluation was another key aspect of the project. Accuracy alone didn’t make sense here, because the dataset is sparse and most movies are unrated by any given user. Instead, I used Precision@K, which measures how many of the top-K recommended movies are actually relevant. Computing this for every user took some time, but it gave a realistic view of the system’s usefulness.

There were a few surprises along the way. Some users’ top recommendations were movies they actually had already rated highly, which made me realize the importance of filtering out already-seen movies. I also noticed that very popular movies tended to appear in many users’ recommendations, which is good in some ways but also risks making the system too “generic.”

The main takeaway for me was that recommendation systems are not just about predicting ratings or showing popular items. It’s about personalizing suggestions in a way that truly helps the user discover new content. Handling missing data, choosing similarity metrics, and evaluating recommendations with the right metrics are just as important as the core algorithm. Overall, this project reinforced that thoughtful preprocessing and evaluation are crucial, and even simple user-based collaborative filtering can produce meaningful recommendations if done carefully.